In [ ]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")

## 1. 處理股價

In [ ]:
stock = pd.read_excel('stock_data_2019-2023.xlsx', sheet_name = '上市2022')

選擇長榮 (2603) 作為此次預測標的

In [ ]:
ase = stock[stock['證券代碼'].str.contains('2603')]
ase['年月日'] = pd.to_datetime(ase['年月日']).dt.date
ase = ase.sort_values(by = '年月日').reset_index(drop = True)

In [ ]:
ase = ase[['證券代碼', '年月日', '收盤價(元)']]
ase['單日漲跌幅(%)'] = ase['收盤價(元)'].rolling(window=2).apply(lambda x: (x.iloc[1]-x.iloc[0])/x.iloc[0])
ase['label'] = ase['單日漲跌幅(%)'].apply(lambda x: (x>0 and '漲') or '跌')

In [ ]:
ase

,證券代碼,年月日,收盤價(元),單日漲跌幅(%),label
0,2603 長榮,2022-01-03,269.2919,NaN,跌
1,2603 長榮,2022-01-04,271.2223,0.007168,漲
2,2603 長榮,2022-01-05,272.1875,0.003559,漲
3,2603 長榮,2022-01-06,276.0483,0.014184,漲
4,2603 長榮,2022-01-07,271.2223,-0.017482,跌
...,...,...,...,...,...
241,2603 長榮,2022-12-26,162.5000,-0.029851,跌
242,2603 長榮,2022-12-27,162.5000,0.000000,跌
243,2603 長榮,2022-12-28,159.0000,-0.021538,跌
244,2603 長榮,2022-12-29,160.0000,0.006289,漲


In [ ]:
ase['label'].value_counts()

跌    131
漲    115
Name: label, dtype: int64

## 2. 處理文章資料

In [ ]:
news19 = pd.read_csv('/Users/apple/Desktop/大數據與商業分析/mid-term report/bda2023_mid_dataset/bda2023_mid_news_2019.csv') # 104466
news20 = pd.read_csv('/Users/apple/Desktop/大數據與商業分析/mid-term report/bda2023_mid_dataset/bda2023_mid_news_2020.csv')
news21 = pd.read_csv('/Users/apple/Desktop/大數據與商業分析/mid-term report/bda2023_mid_dataset/bda2023_mid_news_2021.csv')
news2223 = pd.read_csv('/Users/apple/Desktop/大數據與商業分析/mid-term report/bda2023_mid_dataset/bda2023_mid_news_2022-2023.csv') #  34630
news23 = pd.concat([news19, news20,news21,news2223]).reset_index().drop(["index"], axis=1)
news23

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1546294835402_N01,news,yahoo股市,最新財經新聞,2019-01-01 03:45:00,【歐股盤後】氣氛樂觀 盤勢走穩,中央社 中央社,（中央社台北2019年1月1日電）即將舉行的美中貿易談判為投資人帶來希望，歐洲股市在封關前最...,https://tw.stock.yahoo.com/news/歐股盤後-氣氛樂觀-盤勢走穩...
1,1546293936100_N01,news,yahoo股市,重大要聞,2019-01-01 05:18:00,台股元旦休市期間 美股累計漲跌幅--12月31日,鉅亨網 鉅亨網編譯郭照青,-------------------12 月 27 日 -------12 月 31 日<...,https://tw.stock.yahoo.com/news/台股元旦休市期間-美股累計漲...
2,1546302955899_N01,news,yahoo股市,重大要聞,2019-01-01 05:50:00,2019最受期待10款新車亮相 國產ALTIS和FOCUS成焦點 雙B大型豪華SUV對戰,中時電子報 報導陳大任,中國時報 延續2018競爭激烈的車市氛圍來到2019，今年將有多款新車等著跳上車市擂台一較高...,https://tw.stock.yahoo.com/news/2019最受期待10款新車亮...
3,1546296648699_N01,news,yahoo股市,最新財經新聞,2019-01-01 06:03:00,【美股盤後】封關收紅,中央社 中央社,（中央社台北2019年1月1日電）美股31日在2018年最後一個交易日收漲，不過這是10年前...,https://tw.stock.yahoo.com/news/美股盤後-封關收紅-2203...
4,1546296650082_N01,news,yahoo股市,最新財經新聞,2019-01-01 06:30:00,【能源盤後】年終收紅,中央社 中央社,PR2F3301.DBP.US.GB.OIL.ECO.（中央社台北2019年1月1日電）國際...,https://tw.stock.yahoo.com/news/能源盤後-年終收紅-2230...
...,...,...,...,...,...,...,...,...,...
881341,1679411674978_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 22:28:54,德國3月投資人信心銳減,陳穎芃／綜合外電報導,ZEW經濟研究院公布，3月德國投資人信心指數從2月的28.1點驟降至13.0點，且低於路透調...,https://tw.stock.yahoo.com/news/%E5%BE%B7%E5%9...
881342,1679411670754_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 22:30:06,台中單元二「陸府觀森」兼具ESG與宜居 綠實力成新奢標竿,曾麗芳／台中報導,台中新興隱富聚落、位於北單元二重劃區的「陸府觀森」近期落成，18層樓高的建築外觀猶如一座森林...,https://tw.stock.yahoo.com/news/%E5%8F%B0%E4%B...
881343,1679410774885_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 22:45:45,【公告】安力-KY代子公司昆山新力精密五金取得理財產品,中央社,日 期：2023年03月21日公司名稱：安力-KY (5223)主 旨：代子公司昆山新力精密...,https://tw.stock.yahoo.com/news/%E5%85%AC%E5%9...
881344,1679410774748_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 22:46:51,【公告】安力-KY董事會決議召開股東會相關訊息,中央社,日 期：2023年03月21日公司名稱：安力-KY (5223)主 旨：安力-KY董事會決議...,https://tw.stock.yahoo.com/news/%E5%85%AC%E5%9...


In [ ]:
news23_ase = news23[news23['title'].str.contains('長榮') | news23['content'].str.contains('長榮')].reset_index(drop = True)
news23_ase

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1546387579848_N01,news,yahoo股市,最新財經新聞,2019-01-02 07:50:00,107/12/28上市投信買超排行前20名,Moneydj理財網,"名次 股票名稱 買賣超張數 收盤價 漲跌 買進張數 賣出張數 1 2345智邦 1,273 ...",https://tw.stock.yahoo.com/news/107-12-28上市投信買...
1,1546388491081_N01,news,yahoo股市,最新財經新聞,2019-01-02 07:56:00,【統一證券晨訊】築底完成 攻萬點指日可待,中央社 中央社,日期：2019年 1月 2日<BR>※盤勢分析<BR>新年開春資金回流，技術面轉佳，偏多操作...,https://tw.stock.yahoo.com/news/統-證券晨訊-築底完成-攻萬...
2,1546388494440_N01,news,yahoo股市,最新財經新聞,2019-01-02 07:56:00,投信個股：上週五(28日)投信買超╱賣超個股一覽表,財訊快報 編輯部,"買超張數如下：<BR>代號 個 股 張 數<BR>2345 智 邦 1,273<BR>151...",https://tw.stock.yahoo.com/news/投信個股-上週五-28日-投...
3,1546389403593_N01,news,yahoo股市,最新財經新聞,2019-01-02 08:00:00,《外資》買超股：台積電、中鋼、友達,時報資訊,經統計，前個交易日(20181228)<BR> 1.外資連續買超五日以上的股票：<BR> 正...,https://tw.stock.yahoo.com/news/外資-買超股-台積電-中鋼-...
4,1546391182387_N01,news,yahoo股市,最新財經新聞,2019-01-02 08:47:00,統一證券：台股線型轉佳，新春開紅盤機率高,Moneydj理財網,：<BR> 在美股四大指數尾盤收紅的激勵下，台股上周五以上漲8點開出，雖然外資休假，成交量能...,https://tw.stock.yahoo.com/news/統-證券-台股線型轉佳-新春...
...,...,...,...,...,...,...,...,...,...
15520,1679387461418_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 16:19:47,《台北股市》外資21日賣超前十大 中信金淪ATM、貨櫃三雄全沉,時報資訊,【時報-台北電】瑞信危機暫歇，台股21日小幅開高，惟月線反壓尚未突破，加權指數收15513....,https://tw.stock.yahoo.com/news/%E5%8F%B0%E5%8...
15521,1679387460448_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 16:19:49,《台北股市》三大法人21日賣超前十大 4檔沒「電」遭脫手1萬8000張,時報資訊,【時報-台北電】台股21日加權指數收15513.45點，漲93.48點或0.61%，總成交值...,https://tw.stock.yahoo.com/news/%E5%8F%B0%E5%8...
15522,1679391872095_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 16:27:00,電金齊揚，台股收漲93點、站上10日線,Moneydj理財網,MoneyDJ新聞 2023-03-21 16:27:12 記者 新聞中心 報導市場對歐美銀...,https://tw.stock.yahoo.com/news/%E9%9B%BB%E9%8...
15523,1679393432277_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 17:05:06,有風土有歷史有意思 礁溪長榮鳳凰酒店下午茶蘭陽美食攏底佳,姚舜／台北報導,當宜蘭花生捲遇上哈根達斯冰淇淋會擦出什麼火花？礁溪美人腿與古早味金棗迸出鮮滋味！礁溪長榮鳳凰...,https://tw.stock.yahoo.com/news/%E6%9C%89%E9%A...


* 將第 n 天的文章與第 n+1 的股市漲跌標籤合併，並且只保留我們需要的欄位資訊

In [ ]:
news23_ase['post_time'] = pd.to_datetime(news23_ase['post_time']).dt.date
ase['年月日-1'] = ase['年月日'] - datetime.timedelta(days=1)
news23_ase = pd.merge(news23_ase, ase, left_on='post_time', right_on='年月日-1')[['post_time', 'title', 'content', 'label']]
news23_ase

,post_time,title,content,label
0,2022-01-02,散戶必學四招 40個月年終自己賺,長榮員工爽領40個月、超過百萬元年終獎金的議題，引發全台灣勞工超高的關注度。台股專業分析師表...,跌
1,2022-01-02,16檔新年新趨勢 法人挺,台股2022年元月行情搶先起跑，三大法人卡位包括長榮、合晶、金像電等16檔個股，分別屬於海空...,跌
2,2022-01-02,中再保 航運股幫打氣,中再保（2851）2021年前三季淨利16.53億元，每股盈餘2.8元，獲利穩健。中再保20...,跌
3,2022-01-02,《各報要聞》16檔新年新趨勢 法人挺,【時報-台北電】台股2022年元月行情搶先起跑，三大法人卡位包括長榮、合晶、金像電等16檔個...,跌
4,2022-01-02,《各報要聞》散戶必學四招 40個月年終自己賺,【時報-台北電】長榮員工爽領40個月、超過百萬元年終獎金的議題，引發全台灣勞工超高的關注度。...,跌
...,...,...,...,...
4537,2022-12-29,半導體三雄跌幅收斂 台股萬四失而復得,台股早盤進行恐慌性殺出，加權指數於9點半過後數度摜破萬四大關，隨後在特定買盤持續湧入，萬四失...,漲
4538,2022-12-29,懶人包》2023 四大超商、連鎖藥妝、家樂福、全聯...福袋開賣日、內容！(持續更新),每到新年大家都會購買福袋來試試手氣，四大超商及全聯紛紛推出限定福袋。萊爾富率先開賣三款「開運...,漲
4539,2022-12-29,【台股盤後】台股力守萬四關卡 科技股相對弱勢,（中央社記者江明晏台北2022年12月29日電）台股今天開低走低，早盤一度重挫逾190點，失...,漲
4540,2022-12-29,《台北股市》外資續砍百億 台股今年來跌逾4000點 外資提款1.2兆,【時報-台北電】中國大陸疫情爆發引起恐慌，美股周三(28日)續挫，台股今(29日)早開低走低...,漲


# 3. 訓練集文章向量化處理

* 首先先進行斷詞處理，以下我們會先實作幾個步驟：
    1. 正則表示法清除多餘字元：先移除文章中符號、英數字，只保留中文字元
    2. 斷句：由於 monpa 在處理 200 字以上字串的斷詞時可能會出現錯誤結果，因此我們統一對長文章先進行斷句拆成較短的句子組成的 list
    3. 斷詞：透過 monpa 對斷句結果中的所有句子進行斷詞

In [ ]:
import monpa
from monpa import utils
import re

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [ ]:
# 這個function用來將字串以正則化處理去除中文字元以外的字元
def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

# 我們從stopwords_zh.txt這個檔案中匯入繁體中文的停用詞
with open('/Users/apple/Downloads/stopwords_zh.txt', 'r') as file:
    stopwords = file.read().splitlines()
file.close()

* 我們想嘗試用 2022 年 7 月至 9 月的文章資料來訓練一個分類器，並用於 10 月的股價變動預測當中

In [ ]:
# 設定訓練資料集的開始日期與結束日期
train_startDate = datetime.date(2022,7,1)
train_endDate = datetime.date(2022,9,30)

# 透過monpa對文章進行斷詞處理，並將個別斷詞結果寫在tokenStr這個字串變數中，最後再將訓練集中所有字串存在tokenStr_list中
train_tokenStr_list = []
for i in list(news23_ase[news23_ase['post_time'].between(train_startDate, train_endDate)].index):
    try:
#         txt = clearSentence(news23_ase['content'][i])
        sentence_list = utils.short_sentence(news23_ase['content'][i])
        tokenStr = str()
        for sentence in sentence_list:
            sentence = clearSentence(sentence)
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        train_tokenStr_list.append(tokenStr)
    except:
        train_tokenStr_list.append('')

In [ ]:
train_tokenStr_list

['長榮 海運 周四 除息秀 失靈 當日 股價 雖 一 度 衝高 但 盤 中 湧現 賣壓 終場 跌破 元 往後 兩 天 股價 持續 下探 失守 元 連日 的 下跌 使 不少 水手 套牢 連 同日 才 公告 買進 長榮 的 漢來美食 也 認賠 殺出 虧損 萬 元 看 的 網友 驚呼 原來 是 你 在 殺 漢來 美食日 公告 自 今年 月日 至日 期間 斥資 萬 元 買進 張長榮 股票 成本價 落在 元 至 元 累計 持有 張不過 日 股價 雖 一 度 衝高 至 元 但 盤 中 賣壓 沉重 尾盤 收 在 元後 兩 天 股價 持續 下殺 昨日 收 在 每 股 元 隨著 長榮元 關卡 接連 跌破 漢來美食 再 發布 重訊 處分 張 長榮 股票 均價 落在 元 至 元 處分 損失 約 萬 元 還 持有 張消息 傳至 股板 掀起 熱議 原來 是 你 在 殺 他 不會 是 第一 個 逃 的 也 不會 是 最後 一 個 大 韭菜 被 割 了 終於 撐不住 了 還 有 張 沒 賣 不 是 前天 才 買 這 個 畢業文 有 誠意 吧',
 '時報 台北 電美股 上周 續挫 台股 同步 大跌 並 破底 一 周 跌掉 近 千 點 外資 上周 續 大幅 賣超 台股 達 億 元 外資 賣超 集中 金控股 然 外資 上周 買超 第一 名 是 反 逾 萬 張 明顯 偏空 操作 投信 上周 仍 買超 台股 億 元 買超 二 檔 防禦型 的 電信股 並 與 外資 對作 買超 不少 金控股 美國月 消費 支出 數據 不如 預期 科技股 續挫台股 上周五 在 月 首日 再現 大逃 殺 台積電 聯發科 聯電 領頭 下 殺 貨櫃 三雄 盤 中 跟進 均 跌 逾 半 根 停板 電子 金融 傳產 全面 遭 賣壓 襲擊 加權 指數 開 低 殺低 終場 重挫點 或 收 在 點 續創 逾 一 年 半 新低 成交值 放大 至 億 元 台股 日 及 周 皆 連 四 黑 上周 累計 挫跌點 或台股 繼 月 狂殺點 寫下 最 慘月 紀錄 隨著 經濟 衰退 疑慮 再 升高 美股 挫跌 台股 上周五 再 見 資金 大逃 殺月 開局 就 再度 重跌 外資 單日 賣 超 億 元 本土 的 自營商 及 投信 均 逢低 買進 台股 上周 重挫 以及 今年 以來 大跌 主要 就是 美股 走弱 資金 撤出 外資 上周 賣超 台股 仍 逾 億 

* 文章向量化處理：我們透過 sklearn 套件中 TfidfVectorizer 將斷詞結果去除停用詞後轉為空間向量

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stopwords)
X_train = vectorizer.fit_transform(train_tokenStr_list)
X_train = pd.DataFrame(X_train.toarray(),columns=vectorizer.get_feature_names())
X_train

,一一一億,一七兆,一七月,一三二六五億,一三五,一下子,一九七七億,一二,一併,一兆,...,龍漢翔,龍漢翔寶,龍玉玲,龍航,龍頭,龍頭台積電,龍頭店,龍頭股,龐大,龜山
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.049424,0.0,0.0,0.0,0.0,0.0


* 可以看到使用全部的斷詞結果去組成空間向量時稀疏性會非常大，在後續預測時效率會很低，因此我們需要選擇對分類結果有較顯著影響的詞彙作為向量空間的維度，以下我們透過 Chi-square 計算各詞彙與漲跌標籤的獨立性作為選擇向量空間維度的依據。

In [ ]:
y_train = news23_ase[news23_ase['post_time'].between(train_startDate, train_endDate)]['label']

chi2_selector = SelectKBest(chi2, k = 'all')
chi2_selector.fit(X_train, y_train)
kbest_vocabs = X_train.columns[chi2_selector.get_support()]
X_train = X_train[kbest_vocabs]
X_train

,一一一億,一七兆,一七月,一三二六五億,一三五,一下子,一九七七億,一二,一併,一兆,...,龍漢翔,龍漢翔寶,龍玉玲,龍航,龍頭,龍頭台積電,龍頭店,龍頭股,龐大,龜山
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.049424,0.0,0.0,0.0,0.0,0.0


# 4. 測試集文章向量化處理

* 接著我們可以透過 7-9 月的向量維度將 10 月份的文章也轉成 tf-idf 的向量空間

In [ ]:
# 首先先計算4月份的文章個別的tfidf
test_startDate = datetime.date(2022,10,1)
test_endDate = datetime.date(2022,10,31)

test_tokenStr_list = []
for i in list(news23_ase[news23_ase['post_time'].between(test_startDate, test_endDate)].index):
    try:
        txt = clearSentence(news23_ase['content'][i])
        sentence_list = utils.short_sentence(txt)
        tokenStr = str()
        for sentence in sentence_list:
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        test_tokenStr_list.append(tokenStr)
    except:
        test_tokenStr_list.append('')

In [ ]:
# 接著將4月份的結果透過df.reindex這個方法映射到訓練集的向量空間中
y_test = news23_ase[news23_ase['post_time'].between(test_startDate, test_endDate)]['label']

vectorizer = TfidfVectorizer(stop_words=stopwords)
X_test = vectorizer.fit_transform(test_tokenStr_list)
X_test = pd.DataFrame(X_test.toarray(),columns=vectorizer.get_feature_names())
X_test = X_test.reindex(kbest_vocabs, axis=1, fill_value=0)
X_test

,一一一億,一七兆,一七月,一三二六五億,一三五,一下子,一九七七億,一二,一併,一兆,...,龍漢翔,龍漢翔寶,龍玉玲,龍航,龍頭,龍頭台積電,龍頭店,龍頭股,龐大,龜山
0,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.000000,0,0,0.0,0,0
1,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.000000,0,0,0.0,0,0
2,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.071918,0,0,0.0,0,0
3,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.000000,0,0,0.0,0,0
4,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.069902,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.000000,0,0,0.0,0,0
272,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.045759,0,0,0.0,0,0
273,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.000000,0,0,0.0,0,0
274,0,0,0,0,0,0.0,0,0.0,0.0,0,...,0,0,0,0,0.000000,0,0,0.0,0,0


In [ ]:
news23_ase.to_csv('/Users/apple/Desktop/大數據與商業分析/mid-term report/news23_ase.csv', index=False, encoding="utf-8")